In [1]:
import requests
import os
import bs4 
from time import sleep

In [2]:
def get_directories(passed_html):
    sub_directories = passed_html.find_all("a", class_="js-navigation-open Link--primary")

    repositories = []
    for sub_directory in sub_directories:
        for js_string in str(sub_directory).split('"'):
            if js_string.startswith("/"):
                repositories.append("".join(["https://github.com", js_string]))

    return repositories

In [3]:
http_request = requests.get("https://github.com/SharmaLlama/ticktack/tree/main/src/data/datasets") 
html_parser = bs4.BeautifulSoup(http_request.text, "html.parser")

In [4]:
data_sets = []
for dir in get_directories(html_parser):
    data_set = requests.get(dir)
    passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")

    data_sets.append(get_directories(passed_data))

In [9]:
for i, directory_data_sets in enumerate(data_sets):
    for data_set_url in directory_data_sets:
        if data_set_url.find("csv") == -1:
            data_set = requests.get(data_set_url)
            passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")
            data_sets[i] = get_directories(passed_data)

In [66]:
for data_urls in data_sets:
    for data_url in data_urls:
        data_set = requests.get(data_url)
        data_set = bs4.BeautifulSoup(data_set.text, "html.parser")

        line_number_tags = data_set.find_all("td", class_="blob-num js-line-number")
        number_of_lines = int(line_number_tags[-1]["id"][1:]) - 1

        data = {
            "year": [],
            "d14c": [],
            "sig_d14c": []
        }
        
        tabular_data = data_set.find_all("td")[:-1]
        for line_number in range(number_of_lines * 4):
            column = line_number % 4
            if column == 1:
                pass
            elif column == 2:
                data["year"].append(tabular_data[line_number]) 
            elif column == 3:
                data["d14c"].append(tabular_data[line_number])
            elif column == 0:
                data["sig_d14c"].append(tabular_data[line_number])
